# Logic and Model-Based Reasoning

In [133]:
from utils import *
from logic import *
from copy  import *

**Notes on Logic**

Syntax:

Logical **not** (negation): $\neg$, ~

Logical **and** (conjunction): $\wedge$, &

Logical **or** (disjunction): $\vee$, |

De Morgan’s Theorem

$\neg(A \wedge B) => (\neg A) \vee (\neg B)$

$\neg(A \vee B) => (\neg A) \wedge (\neg B)$

...

# ## Practice reducing sentances to CNF
The cell below will reduce logical sentances to CNF.  This is useful to check practice problems.  The following syntax is used:

Implies: ==>, >>

IFF: <=>

and: &

not: ~

or: |

The flag SHOW_STEPS controls how much of the process is displayed. 

SHOW_STEPS = False - Output is only final answer

SHOW_STEPS = True - Output shows each step in the reduction process

In [13]:
#Examples
s = "A==>B"
to_cnf(s)

(B | ~A)

In [14]:
s = "A<=>B"
to_cnf(s)

((A | ~B) & (B | ~A))

In [15]:
s = "(A&B)|(C&D)"
to_cnf(s)

((C | A) & (D | A) & (C | B) & (D | B))

In [16]:
s = "(A<=>B)&(B==>C)"
to_cnf(s, True)

Initial Expression:
((A <=> B) & (B >> C))
Eliminate Implications:
(((A | ~B) & (B | ~A)) & (C | ~B))
Distribution
((A | ~B) & (B | ~A) & (C | ~B))


((A | ~B) & (B | ~A) & (C | ~B))

In [17]:
s = "(A<=>B)|~(D&C)"
to_cnf(s, True)

Initial Expression:
((A <=> B) | ~(D & C))
Eliminate Implications:
(((A | ~B) & (B | ~A)) | ~(D & C))
De Morgan's Theorem:
(((A | ~B) & (B | ~A)) | (~D | ~C))
Distribution
((A | ~B | ~D | ~C) & (B | ~A | ~D | ~C))


((A | ~B | ~D | ~C) & (B | ~A | ~D | ~C))

In [ ]:
#HW 9 Problems

In [ ]:
#B.1
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

In [20]:
#B.2
s = "A<=>A"
to_cnf(s, SHOW_STEPS)

Initial Expression:
(A <=> A)
Eliminate Implications:
((A | ~A) & (A | ~A))


((A | ~A) & (A | ~A))

In [21]:
#B.3
s = "(A<=>B)|C"
to_cnf(s, SHOW_STEPS)

Initial Expression:
((A <=> B) | C)
Eliminate Implications:
(((A | ~B) & (B | ~A)) | C)
Distribution
((A | ~B | C) & (B | ~A | C))


((A | ~B | C) & (B | ~A | C))

In [138]:
SHOW_STEPS = True
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

Initial Expression:
~((~A & B) | (C & D))
De Morgan's Theorem:
((A | ~B) & (~C | ~D))


((A | ~B) & (~C | ~D))

In [139]:
#Demonstrate the steps of the unit propogation

In [191]:
#Split a sentance into clauses
s = "~((~A&B)|(~C&D)|C|B)"
clauses = conjuncts(to_cnf(s))
to_cnf(s)

((A | ~B) & (C | ~D) & ~C & ~B)

In [192]:
#To find a valid model of the sentance, all clauses must be true
#Look for clauses with single values, and add their negations to the list of expressions that must be false
false_exp = []
true_exp = []
for c in clauses:
    if len(c.args)<=1:
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        print false_exp

[C]
[C, B]


In [193]:
#Remove already satisfied clases
for i in true_exp:
    clauses.remove(i)

print clauses

[(A | ~B), (C | ~D)]


In [194]:
cp_clauses = deepcopy(clauses)
#Remove all literals known to be false from the other clases
for c in cp_clauses:
    for fe in false_exp:
        if fe in deepcopy(c.args):
            c.args.remove(fe)

print cp_clauses

#If any clauses are remaining with a single literal, add that literal to the known true/fault lists repeat
print false_exp
for c in cp_clauses:
    print c
    repeat = False
    if len(c.args) <= 1:
        if len(c.args) == 1:
            c =  expr(c.args[0].__repr__())
            print c
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        repeat = True

print true_exp
        


            

[(A | ~B), |~D]
[C, B]
(A | ~B)
|~D
~D
[~C, ~B, ~D]
